<a href="https://colab.research.google.com/github/mafemunoz/Trabajo-IA-2025-2/blob/main/02%20-%20preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Carga el archivo kaggle.json
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!python -m pip install -qq kaggle
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia
!unzip udea-ai-4-eng-20252-pruebas-saber-pro-colombia

Saving kaggle (1).json to kaggle (1) (1).json
udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# 01 - exploración.ipynb
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Cargar datos
train = pd.read_csv('train.csv')

In [ ]:
# estos son los primeros datos de las columnas del datafarme
print(train.shape)
train.head()

(692500, 21)


,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


In [ ]:
#información basica de dataframe
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 21 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   ID                           692500 non-null  int64  
 1   PERIODO_ACADEMICO            692500 non-null  int64  
 2   E_PRGM_ACADEMICO             692500 non-null  object 
 3   E_PRGM_DEPARTAMENTO          692500 non-null  object 
 4   E_VALORMATRICULAUNIVERSIDAD  686213 non-null  object 
 5   E_HORASSEMANATRABAJA         661643 non-null  object 
 6   F_ESTRATOVIVIENDA            660363 non-null  object 
 7   F_TIENEINTERNET              665871 non-null  object 
 8   F_EDUCACIONPADRE             669322 non-null  object 
 9   F_TIENELAVADORA              652727 non-null  object 
 10  F_TIENEAUTOMOVIL             648877 non-null  object 
 11  E_PRIVADO_LIBERTAD           692500 non-null  object 
 12  E_PAGOMATRICULAPROPIO        686002 non-null  object 
 13 

In [ ]:
# Contar los NaN por columna
na_por_columna = train.isna().sum()
print(na_por_columna)

ID                                 0
PERIODO_ACADEMICO                  0
E_PRGM_ACADEMICO                   0
E_PRGM_DEPARTAMENTO                0
E_VALORMATRICULAUNIVERSIDAD     6287
E_HORASSEMANATRABAJA           30857
F_ESTRATOVIVIENDA              32137
F_TIENEINTERNET                26629
F_EDUCACIONPADRE               23178
F_TIENELAVADORA                39773
F_TIENEAUTOMOVIL               43623
E_PRIVADO_LIBERTAD                 0
E_PAGOMATRICULAPROPIO           6498
F_TIENECOMPUTADOR              38103
F_TIENEINTERNET.1              26629
F_EDUCACIONMADRE               23664
RENDIMIENTO_GLOBAL                 0
INDICADOR_1                        0
INDICADOR_2                        0
INDICADOR_3                        0
INDICADOR_4                        0
dtype: int64


In [ ]:
# observar si alguno de los datos categoricos tiene un solo subnivel y eliminarlo, ya que no es significativo en ese caso
cols_cat =['E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO', 'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA',
 'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE', 'F_TIENELAVADORA',
 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD', 'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR',
 'F_TIENEINTERNET.1', 'F_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL']
for col in cols_cat:
  print(f'Columna {col}: {train[col].nunique()} subniveles')
# por este metodo, no se elimina ninguna columna

# observar las categorias númericas
train.describe()
# la desviación no es cero, por este motovo no se elimina ninguna columna

Columna E_PRGM_ACADEMICO: 948 subniveles
Columna E_PRGM_DEPARTAMENTO: 31 subniveles
Columna E_VALORMATRICULAUNIVERSIDAD: 8 subniveles
Columna E_HORASSEMANATRABAJA: 5 subniveles
Columna F_ESTRATOVIVIENDA: 7 subniveles
Columna F_TIENEINTERNET: 2 subniveles
Columna F_EDUCACIONPADRE: 12 subniveles
Columna F_TIENELAVADORA: 2 subniveles
Columna F_TIENEAUTOMOVIL: 2 subniveles
Columna E_PRIVADO_LIBERTAD: 2 subniveles
Columna E_PAGOMATRICULAPROPIO: 2 subniveles
Columna F_TIENECOMPUTADOR: 2 subniveles
Columna F_TIENEINTERNET.1: 2 subniveles
Columna F_EDUCACIONMADRE: 12 subniveles
Columna RENDIMIENTO_GLOBAL: 4 subniveles


,ID,PERIODO_ACADEMICO,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
count,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000
mean,494606.130576,20198.366679,0.268629,0.259996,0.262087,0.262903
std,285585.209455,10.535037,0.122130,0.093480,0.058862,0.067944
min,1.000000,20183.000000,0.000000,0.000000,0.000000,0.000000
25%,247324.750000,20195.000000,0.203000,0.212000,0.254000,0.255000
50%,494564.500000,20195.000000,0.240000,0.271000,0.276000,0.285000
75%,741782.500000,20203.000000,0.314000,0.309000,0.293000,0.303000
max,989286.000000,20213.000000,0.657000,0.487000,0.320000,0.332000


In [ ]:
#observar el contenido de de las columnas
category_cols = [i for i in train.columns if not i in train._get_numeric_data()]
for c in category_cols:
    print ("%10s"%c, np.unique(train[c].dropna()))

E_PRGM_ACADEMICO ['3°  CICLO PROFESIONAL NEGOCIOS INTERNACIONALES'
 'ACTIVIDAD FISICA Y DEPORTE' 'ACUICULTURA' 'ADMINISTRACION'
 'ADMINISTRACION  FINANCIERA' 'ADMINISTRACION & SERVICIO'
 'ADMINISTRACION AERONAUTICA' 'ADMINISTRACION AGROPECUARIA'
 'ADMINISTRACION AMBIENTAL'
 'ADMINISTRACION AMBIENTAL Y DE LOS RECURSOS NATURALES'
 'ADMINISTRACION BANCARIA Y FINANCIERA' 'ADMINISTRACION COMERCIAL'
 'ADMINISTRACION COMERCIAL Y DE MERCADEO'
 'ADMINISTRACION DE COMERCIO EXTERIOR' 'ADMINISTRACION DE EMPRESAS'
 'ADMINISTRACION DE EMPRESAS  Y  GESTION AMBIENTAL'
 'ADMINISTRACION DE EMPRESAS AGROINDUSTRIALES'
 'ADMINISTRACION DE EMPRESAS AGROPECUARIAS'
 'ADMINISTRACION DE EMPRESAS COMERCIALES'
 'ADMINISTRACION DE EMPRESAS EN TELECOMUNICACIONES'
 'ADMINISTRACION DE EMPRESAS TURISTICA'
 'ADMINISTRACION DE EMPRESAS TURISTICAS'
 'ADMINISTRACION DE EMPRESAS TURISTICAS Y HOTELERAS'
 'ADMINISTRACION DE EMPRESAS TURÍSTICAS Y HOTELERAS'
 'ADMINISTRACION DE EMPRESAS Y FINANZAS'
 'ADMINISTRACION DE GESTISN 

la columna E_PRIVADO_LIBERTAD, solo tiene el componente no, lo cual reduce la capacidad del model en procesos analíticos o predictivos. El objetivo principal de este análisis está enfocado en evaluar factores académicos, socioeconómicos y demográficos con impacto generalizado. La condición de privación de libertad, si bien relevante en contextos específicos, no representa una característica común o directamente relacionada con los resultados educativos de la población general estudiada. por tanto esta colum ser eliminada y se hara un nuevo dataframe

para F_TIENEINTERNET.1, se observa que ya existe una columna con esta información, por ende se eliminara

para INDICADOR_1, INDICADOR_2, INDICADOR_3, INDICADOR_4. no se tiene información suficiente sobre lo que significan estas variables, ya sea por la informaicón suministrada en la pagina de la competencia, que no indica lo que significan o por el nombre de la varaible, se ha decidida eliminar estas columnas

In [ ]:
#se eliminan las columnas
train = train.drop(labels =['E_PRIVADO_LIBERTAD','F_TIENEINTERNET.1', 'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4'],axis= 1)

Ahora vamos a renombrar las columnas, para que sea mas coherente con su contenido

In [ ]:
train.columns = ['ID','PERIODO','PROGRAMA','DEPARTAMENTO','VALOR_MATRICULA','HORAS_TRABAJA','ESTRATO','TIENE_INTERNET','EDUCACION_PADRE','TIENE_LAVADORA','TIENE_AUTO','PAGA_MATRICULA','TIENE_COMPUTADOR','EDUCACION_MADRE','RENDIMIENTO_GLOBAL']

print(train.columns)

Index(['ID', 'PERIODO', 'PROGRAMA', 'DEPARTAMENTO', 'VALOR_MATRICULA',
       'HORAS_TRABAJA', 'ESTRATO', 'TIENE_INTERNET', 'EDUCACION_PADRE',
       'TIENE_LAVADORA', 'TIENE_AUTO', 'PAGA_MATRICULA', 'TIENE_COMPUTADOR',
       'EDUCACION_MADRE', 'RENDIMIENTO_GLOBAL'],
      dtype='object')


In [ ]:
# crear una función para convertir en binario
def convertir_num(valor):
    if valor == 'Si':
        return 1
    elif valor == 'No':
        return 0
    else:
        return valor  # Si no es 'Si' ni 'No', dejar el valor original

# Aplicar la función a todas las columnas del DataFrame
train = train.applymap(convertir_num)

/tmp/ipython-input-3458156645.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train = train.applymap(convertir_num)


In [ ]:
# vamos a convertir a variables categoricas las columnas que se identificaron tienen estas caracteristicas
valores_rendimiento = {'alto': 4, 'medio-alto': 3, 'medio-bajo': 2, 'bajo': 1}
train["RENDIMIENTO_GLOBAL"] = train["RENDIMIENTO_GLOBAL"].replace(valores_rendimiento)

/tmp/ipython-input-3765320887.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train["RENDIMIENTO_GLOBAL"] = train["RENDIMIENTO_GLOBAL"].replace(valores_rendimiento)


In [ ]:
# convertir las otras variables categoricas
Columnas_categoricas = ["VALOR_MATRICULA","PROGRAMA","DEPARTAMENTO","EDUCACION_MADRE","EDUCACION_PADRE"]

train[Columnas_categoricas] = train[Columnas_categoricas].astype("category")

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   ID                  692500 non-null  int64   
 1   PERIODO             692500 non-null  int64   
 2   PROGRAMA            692500 non-null  category
 3   DEPARTAMENTO        692500 non-null  category
 4   VALOR_MATRICULA     686213 non-null  category
 5   HORAS_TRABAJA       661643 non-null  object  
 6   ESTRATO             660363 non-null  object  
 7   TIENE_INTERNET      665871 non-null  float64 
 8   EDUCACION_PADRE     669322 non-null  category
 9   TIENE_LAVADORA      652727 non-null  float64 
 10  TIENE_AUTO          648877 non-null  float64 
 11  PAGA_MATRICULA      686002 non-null  float64 
 12  TIENE_COMPUTADOR    654397 non-null  float64 
 13  EDUCACION_MADRE     668836 non-null  category
 14  RENDIMIENTO_GLOBAL  692500 non-null  int64   
dtypes: category(5), f

In [ ]:
# ahora vamos a dejar el estrato solo con sus valores númericos
train['ESTRATO'] = train['ESTRATO'].str.replace('Estrato ', '')

In [ ]:
# reemplazar los valores NaN de las columnas por la moda
columnas_binarias = ['TIENE_INTERNET', 'TIENE_LAVADORA', 'TIENE_AUTO',
                      'PAGA_MATRICULA', 'TIENE_COMPUTADOR']

for columna in columnas_binarias:
    moda = train[columna].mode()[0]
    train[columna] = train[columna].fillna(moda)

In [ ]:
train['ESTRATO'] = train['ESTRATO'].fillna(train['ESTRATO'].mode()[0])
train['EDUCACION_PADRE'] = train['EDUCACION_PADRE'].fillna(train['EDUCACION_PADRE'].mode()[0])
train['EDUCACION_MADRE'] = train['EDUCACION_MADRE'].fillna(train['EDUCACION_MADRE'].mode()[0])
train['VALOR_MATRICULA'] = train['VALOR_MATRICULA'].fillna(train['VALOR_MATRICULA'].mode()[0])
train['HORAS_TRABAJA'] = train['HORAS_TRABAJA'].fillna(train['HORAS_TRABAJA'].mode()[0])

In [ ]:
# verificar que no hay mas datos faltantes
print(train.isnull().sum())

ID                    0
PERIODO               0
PROGRAMA              0
DEPARTAMENTO          0
VALOR_MATRICULA       0
HORAS_TRABAJA         0
ESTRATO               0
TIENE_INTERNET        0
EDUCACION_PADRE       0
TIENE_LAVADORA        0
TIENE_AUTO            0
PAGA_MATRICULA        0
TIENE_COMPUTADOR      0
EDUCACION_MADRE       0
RENDIMIENTO_GLOBAL    0
dtype: int64


In [ ]:
# realizar one hot a la columna de valor matriculo
codificador = OneHotEncoder()

Columnas_OneHot=['VALOR_MATRICULA']
codificacion = codificador.fit_transform(train[Columnas_OneHot])

train_codificado = pd.DataFrame(codificacion.toarray(), columns=codificador.get_feature_names_out(Columnas_OneHot))

train = pd.concat([train,train_codificado], axis = 1 )

train = train.drop(labels =['VALOR_MATRICULA'],axis= 1)

In [ ]:
# realizar one hot a la columna de horas trabaja
codificador = OneHotEncoder()

Columnas_OneHot=['HORAS_TRABAJA']
codificacion = codificador.fit_transform(train[Columnas_OneHot])

train_codificado = pd.DataFrame(codificacion.toarray(), columns=codificador.get_feature_names_out(Columnas_OneHot))

train = pd.concat([train,train_codificado], axis = 1 )

train = train.drop(labels =['HORAS_TRABAJA'],axis= 1)

In [ ]:
#Ordenas final por indice
train2 = train.copy()
train3 = train.copy()
train = train.sort_index(axis=0)
train

,ID,PERIODO,PROGRAMA,DEPARTAMENTO,ESTRATO,TIENE_INTERNET,EDUCACION_PADRE,TIENE_LAVADORA,TIENE_AUTO,PAGA_MATRICULA,...,VALOR_MATRICULA_Entre 5.5 millones y menos de 7 millones,VALOR_MATRICULA_Entre 500 mil y menos de 1 millón,VALOR_MATRICULA_Menos de 500 mil,VALOR_MATRICULA_Más de 7 millones,VALOR_MATRICULA_No pagó matrícula,HORAS_TRABAJA_0,HORAS_TRABAJA_Entre 11 y 20 horas,HORAS_TRABAJA_Entre 21 y 30 horas,HORAS_TRABAJA_Menos de 10 horas,HORAS_TRABAJA_Más de 30 horas
0,904256,20212,ENFERMERIA,BOGOTÁ,3,1.0,Técnica o tecnológica incompleta,1.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,645256,20212,DERECHO,ATLANTICO,3,0.0,Técnica o tecnológica completa,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,3,1.0,Secundaria (Bachillerato) completa,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,4,1.0,No sabe,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,989032,20212,PSICOLOGIA,ANTIOQUIA,3,1.0,Primaria completa,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,20195,BIOLOGIA,LA GUAJIRA,2,1.0,Secundaria (Bachillerato) completa,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
692496,754213,20212,PSICOLOGIA,NORTE SANTANDER,3,1.0,Primaria incompleta,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
692497,504185,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,3,1.0,Secundaria (Bachillerato) completa,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
692498,986620,20195,PSICOLOGIA,TOLIMA,1,0.0,Primaria completa,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
print(train.isnull().sum())

ID                                                          0
PERIODO                                                     0
PROGRAMA                                                    0
DEPARTAMENTO                                                0
ESTRATO                                                     0
TIENE_INTERNET                                              0
EDUCACION_PADRE                                             0
TIENE_LAVADORA                                              0
TIENE_AUTO                                                  0
PAGA_MATRICULA                                              0
TIENE_COMPUTADOR                                            0
EDUCACION_MADRE                                             0
RENDIMIENTO_GLOBAL                                          0
VALOR_MATRICULA_Entre 1 millón y menos de 2.5 millones      0
VALOR_MATRICULA_Entre 2.5 millones y menos de 4 millones    0
VALOR_MATRICULA_Entre 4 millones y menos de 5.5 millones    0
VALOR_MA